In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os,sys
import glob

### Check the directory

In [53]:
# im_path_train = r"C:/Users/Kerem/Desktop/nn_final/ims/train/*.jpg" 
# im_path_test = r"C:/Users/Kerem/Desktop/nn_final/ims/test/*.jpg"
im_path_train = r"/media/seibutsugakun/6C1C500A1C4FCDB0/Users/Kerem/Desktop/nn_final/ims/train/*.jpg"
im_path_test = r"/media/seibutsugakun/6C1C500A1C4FCDB0/Users/Kerem/Desktop/nn_final/ims/test/*.jpg"


### Check Pictures

#### Below code will show all pictures in the given path. In this case , it will search for those ending with .jpg

In [54]:
my_path = r"C:/Users/Kerem/Desktop/nn_final/ims/*.jpg"

In [55]:
image_list = []
for filename in glob.glob(my_path):
    im=Image.open(filename)
    im.show()
    image_list.append(im)

In [56]:
for i in image_list:
    a = np.asarray(i)
    print("image size = ",a.shape)
    plt.imshow(i)

### Get minibatch train data

In [99]:
train_image_list = []
minibatch_size = 200
no_of_images = len(glob.glob(im_path_train)) #75613

current_batch_names = glob.glob(im_path_train)[0:minibatch_size]
current_batch = np.zeros((minibatch_size,120,120,3))

for idx,im in enumerate(current_batch_names):
    current_batch[idx,:,:,:] = np.asarray(Image.open(im))

current_batch.shape

# for filename in glob.glob(im_path_train):
#     im=Image.open(filename)
#     train_image_list.append(im)

ValueError: could not broadcast input array from shape (120,120) into shape (120,120,3)

# Builiding Convnet

In [ ]:
def conv_(img, conv_filter):
    # summary: gets called on each filter and each channel (RGB)
    filter_size = conv_filter.shape[1] #i.e. 3
    result = np.zeros((img.shape[0:2]))
    #Looping through the image to apply the convolution operation.
    for r in np.uint16(np.arange(filter_size/2.0, 
                          img.shape[0]-filter_size/2.0+1)):
        for c in np.uint16(np.arange(filter_size/2.0, 
                                           img.shape[1]-filter_size/2.0+1)):
            """
            Getting the current region to get multiplied with the filter.
            How to loop through the image and get the region based on 
            the image and filter sizes is the most tricky part of convolution.
            """
            curr_region = img[r-np.uint16(np.floor(filter_size/2.0)):r+np.uint16(np.ceil(filter_size/2.0)), 
                              c-np.uint16(np.floor(filter_size/2.0)):c+np.uint16(np.ceil(filter_size/2.0))]
            #Element-wise multipliplication between the current region and the filter.
            curr_result = curr_region * conv_filter
            conv_sum = np.sum(curr_result) #Summing the result of multiplication.
            result[r, c] = conv_sum #Saving the summation in the convolution layer feature map.
            
    #Clipping the outliers of the result matrix.
    final_result = result[np.uint16(filter_size/2.0):result.shape[0]-np.uint16(filter_size/2.0), 
                          np.uint16(filter_size/2.0):result.shape[1]-np.uint16(filter_size/2.0)]
    return final_result

In [ ]:
def conv(img, conv_filter):
    if len(img.shape) > 2 or len(conv_filter.shape) > 3: # Check if number of image channels matches the filter depth.
        if img.shape[-1] != conv_filter.shape[-1]:
            print("Error: Number of channels in both image and filter must match.")
            sys.exit()
    if conv_filter.shape[1] != conv_filter.shape[2]: # Check if filter dimensions are equal.
        print('Error: Filter must be a square matrix. I.e. number of rows and columns must match.')
        sys.exit()
    if conv_filter.shape[1]%2==0: # Check if filter diemnsions are odd.
        print('Error: Filter must have an odd size. I.e. number of rows and columns must be odd.')
        sys.exit()

    # An empty feature map to hold the output of convolving the filter(s) with the image.
    feature_maps = np.zeros((img.shape[0]-conv_filter.shape[1]+1, 
                                img.shape[1]-conv_filter.shape[1]+1, 
                                conv_filter.shape[0]))

    # Convolving the image by the filter(s).
    for filter_num in range(conv_filter.shape[0]):
        print("Filter ", filter_num + 1)
        curr_filter = conv_filter[filter_num, :] # getting a filter from the bank.
        """ 
        Checking if there are mutliple channels for the single filter.
        If so, then each channel will convolve the image.
        The result of all convolutions are summed to return a single feature map.
        """
# #         print("current filter shape is ",curr_filter.shape)
#         if img.shape[-1] >= 2:
#             for depth in range(curr_filter.shape[-1]):
#                 conv_map = conv_(img[:, :, 0], curr_filter(:,:,:,depth)) # Array holding the sum of all feature maps.
#                 for ch_num in range(1,img.shape[-1]): # Convolving each channel with the image and summing the results.
#                     conv_map = conv_map + conv_(img[:, :, ch_num], 
#                                       curr_filter)
                    
        if len(curr_filter.shape) > 2:  
            conv_map = conv_(img[:, :, 0], curr_filter[:, :, 0]) # Array holding the sum of all feature maps.  
            for ch_num in range(1, curr_filter.shape[-1]): # Convolving each channel with the image and summing the results.  
                 conv_map = conv_map + conv_(img[:, :, ch_num],   
                                   curr_filter[:, :, ch_num])  
                       
                    
                    
                    
        else: # There is just a single channel in the filter.
            conv_map = conv_(img, curr_filter)
        feature_maps[:, :, filter_num] = conv_map # Holding feature map with the current filter.
    return feature_maps # Returning all feature maps.

In [ ]:
def pooling(feature_map, size=2, stride=2):
    #Preparing the output of the pooling operation.
    pool_out = np.zeros((np.uint16((feature_map.shape[0]-size+1)/stride+1),
                            np.uint16((feature_map.shape[1]-size+1)/stride+1),
                            feature_map.shape[-1]))
    for map_num in range(feature_map.shape[-1]):
        r2 = 0
        for r in np.arange(0,feature_map.shape[0]-size+1, stride):
            c2 = 0
            for c in np.arange(0, feature_map.shape[1]-size+1, stride):
                pool_out[r2, c2, map_num] = np.max([feature_map[r:r+size,  c:c+size, map_num]])
                c2 = c2 + 1
            r2 = r2 +1
    return pool_out

In [ ]:
def relu(feature_map):
    #Preparing the output of the ReLU activation function.
    relu_out = np.zeros(feature_map.shape)
    for map_num in range(feature_map.shape[-1]):
        for r in np.arange(0,feature_map.shape[0]):
            for c in np.arange(0, feature_map.shape[1]):
                relu_out[r, c, map_num] = np.max([feature_map[r, c, map_num], 0])
    return relu_out

In [ ]:
def spatial_batchnorm():
        
    #for CNN:
    obj.previousInput = input;
    [H,W,C,N] = size(input);

    # Permute the dimensions to the following format 
    # (cols, channel, rows, batch)    
    # Python tensor format:
    # (batch(0), channel(1), rows(2), cols(3))
    xT = x.transpose((0,2,3,1))
    
    #Flat the input (On python the reshape is row-major)           
    
#     inputFlat = reshape_row_major(inputTransposed,[(numel(inputTransposed) / C),C]);
    inputFlat = flatten(xT)
    
    #Call the forward propagation of normal batchnorm
    activations = obj.normalBatchNorm.ForwardPropagation(inputFlat, weights, bias);
    
    #Reshape/transpose back the signal, on python was (N,H,W,C)
    activations_reshape = activations.transpose((N,H,W,C))
    
    #On python was transpose(0,3,1,2)
    activations = np.transpose(activations_reshape,[3 1 2 4]);

    #Store stuff for backpropagation
    obj.activations = activations;
    obj.weights = weights;
    obj.biases = bias;
    
    mode = bn_param['mode']
    epsilon=1e-6
    momentum = bn_param.get('momentum',0.9)
    
    N,D = x.shape
    running_mean = bn_param.get('running_mean',np.zeros(D,dtype=x.dtype))
    running_var = bn_param.get('running_var',np.zeros(D,dtype=x.dtype))

In [74]:
def batchnorm(x,gamma,beta,bn_param):
    #comes before relu // different for FC and CNN (needs reshape)

    mode = bn_param['mode']
    epsilon = bn_param.get('eps',1e-5)
    momentum = bn_param.get('momentum',0.9)
    
    N,D = x.shape
    running_mean = bn_param.get('running_mean',np.zeros(D,dtype=x.dtype))
    running_var = bn_param.get('running_var',np.zeros(D,dtype=x.dtype))
    
    out,cache = None,None
    
    if mode == "train":
        
        mu = 1/ float(N) * np.sum(x, axis=0)
        xmu = x - mu
        carre = xmu**2
        var = 1/ float(N) * np.sum(carre,axis=0)
        sqrtvar = np.sqrt(var + epsilon)
        invvar = 1. / sqrtvar
        va2 = xmu * invvar
        va3 = gamma * va2
        out = va3 + beta
        
        running_mean = momentum * running_mean +(1.0 - momentum) * mu
        running_var = momentum * running_var + (1.0 - momentum) * var
        #store values
        cache = (mu,xmu,carre,var,sqrtvar,invvar,va2,va3,gamma,beta,x,bn_param)
    
    elif mode == "test":
        running_mean = bn_param['running_mean']
        running_var = bn_param['running_var']
        xbar = (x-running_mean) /np.sqrt(running_var+epsilon)
        out = gamma * xbar + beta
        cache = (x,xbar,gamma,beta,epsilon)
    else:
        raise ValueError('Invalid forward batchnorm mode')
    bn_param['running_mean'] = running_mean
    bn_param['running_var'] = running_var
    
    return out,cache

### Preparing Filters

In [60]:
l1_filter = np.zeros((2,3,3)) # 2 filters with 3x3 2D-convolution

#### Set first filters to look for edges

In [61]:
l1_filter[0, :, :] = np.array([[[-1, 0, 1],   
                                     [-1, 0, 1],   
                                     [-1, 0, 1]]])  
l1_filter[1, :, :] = np.array([[[1,   1,  1],   
                                     [0,   0,  0],   
                                     [-1, -1, -1]]]) 

### Now apply multiple convolutions

In [84]:
def stack_cnn_layers(img,l1_filter,gamma=0,beta=0.01,is_training=True):    
    print("Input shape:",img.shape)
    
    num_of_filters = 2
    kernel_size = [3,3]
    #First conv layer. Fixed: Looks for edges                                                 
    print("\n**Working with conv layer 1**",end="\t")
    print("Number of Filters = {}  Kernel size = {}x{}".format(num_of_filters,kernel_size[0],kernel_size[1]))
  
    l1_feature_map = conv(img, l1_filter)
    print("Shape: ",l1_feature_map.shape )
        
    print("\n**ReLU**")
    l1_feature_map_relu = relu(l1_feature_map)
    print("Shape: ",l1_feature_map_relu.shape )

#     print("\n**Pooling**")
#     l1_feature_map_relu_pool = pooling(l1_feature_map_relu, 2, 2)
#     print("Shape: ",l1_feature_map_relu_pool.shape )
    print("**End of conv layer 1**\n")

    # Second conv layer
    num_of_filters = 64
    kernel_size = [5,5]
    
    l2_filter = np.random.rand(num_of_filters, kernel_size[0], kernel_size[1], l1_feature_map_relu.shape[-1])
    print("\n**Working with conv layer 2**",end="\t")
    print("Number of Filters = {}  Kernel size = {}x{}".format(num_of_filters,kernel_size[0],kernel_size[1]))
    l2_feature_map = conv(l1_feature_map_relu, l2_filter)
    print("Shape: ",l2_feature_map.shape )

    print("\n**ReLU**")
    l2_feature_map_relu = relu(l2_feature_map)
    print("Shape: ",l2_feature_map_relu.shape )

    print("\n**Pooling**")
    l2_feature_map_relu_pool = pooling(l2_feature_map_relu, 2, 2)
    print("Shape: ",l2_feature_map_relu_pool.shape )
    
    print("**End of conv layer 2**\n")

    # Third conv layer
    num_of_filters = 128
    kernel_size = [7,7]
    
    l3_filter = np.random.rand(num_of_filters, kernel_size[0], kernel_size[1], l2_feature_map_relu_pool.shape[-1])
    print("\n**Working with conv layer 3**",end="\t")
    print("Number of Filters = {}  Kernel size = {}x{}".format(num_of_filters,kernel_size[0],kernel_size[1]))
    l3_feature_map = conv(l2_feature_map_relu_pool, l3_filter)
    print("Shape: ",l3_feature_map.shape )
    
    print("\n**ReLU**")
    l3_feature_map_relu = relu(l3_feature_map)
    print("Shape: ",l3_feature_map_relu.shape )

#     print("\n**Pooling**")
#     l3_feature_map_relu_pool = pooling(l3_feature_map_relu, 2, 2)
#     print("Shape: ",l3_feature_map_relu_pool.shape )

    print("**End of conv layer 3**\n")
    
    
    # Fourth conv layer
    num_of_filters = 256
    kernel_size = [5,5]
    
    l4_filter = np.random.rand(num_of_filters, kernel_size[0], kernel_size[1], l3_feature_map_relu.shape[-1])
    print("\n**Working with conv layer 4**",end="\t")
    print("Number of Filters = {}  Kernel size = {}x{}".format(num_of_filters,kernel_size[0],kernel_size[1]))
    l4_feature_map = conv(l3_feature_map_relu, l4_filter)
    print("Shape: ",l4_feature_map.shape )
    
    print("\n**ReLU**")
    l4_feature_map_relu = relu(l4_feature_map)
    print("Shape: ",l4_feature_map_relu.shape )

    print("\n**Pooling**")
    l4_feature_map_relu_pool = pooling(l4_feature_map_relu, 2, 2)
    print("Shape: ",l4_feature_map_relu_pool.shape )

    print("**End of conv layer 4**\n")    
    
    
    # Fifth conv layer
    num_of_filters = 512
    kernel_size = [3,3]
    
    l5_filter = np.random.rand(num_of_filters, kernel_size[0], kernel_size[1], l4_feature_map_relu_pool.shape[-1])
    print("\n**Working with conv layer 4**",end="\t")
    print("Number of Filters = {}  Kernel size = {}x{}".format(num_of_filters,kernel_size[0],kernel_size[1]))
    l5_feature_map = conv(l4_feature_map_relu_pool, l5_filter)
    print("Shape: ",l5_feature_map.shape )
    
    print("\n**ReLU**")
    l5_feature_map_relu = relu(l5_feature_map)
    print("Shape: ",l5_feature_map_relu.shape )

#     print("\n**Pooling**")-
#     l5_feature_map_relu_pool = pooling(l4_feature_map_relu, 2, 2)
#     print("Shape: ",l4_feature_map_relu_pool.shape )

    
    print("**End of conv layer 5**\n")    
    last_feature_map = l5_feature_map_relu
    return last_feature_map

In [85]:
# last_feature_map = stack_cnn_layers(img,l1_filter)

In [86]:
minibatch_l1_feature_map = []
for img in current_batch[0:1]:
    last_feature_map = stack_cnn_layers(img,l1_filter)

Input shape: (120, 120, 3)

**Working with conv layer 1**	Number of Filters = 2  Kernel size = 3x3
Filter  1
Filter  2
Shape:  (118, 118, 2)

**ReLU**
Shape:  (118, 118, 2)
**End of conv layer 1**


**Working with conv layer 2**	Number of Filters = 64  Kernel size = 5x5
Filter  1
Filter  2
Filter  3
Filter  4
Filter  5
Filter  6
Filter  7
Filter  8
Filter  9
Filter  10
Filter  11
Filter  12
Filter  13
Filter  14
Filter  15
Filter  16
Filter  17
Filter  18
Filter  19
Filter  20
Filter  21
Filter  22
Filter  23
Filter  24
Filter  25
Filter  26
Filter  27
Filter  28
Filter  29
Filter  30
Filter  31
Filter  32
Filter  33
Filter  34
Filter  35
Filter  36
Filter  37
Filter  38
Filter  39
Filter  40
Filter  41
Filter  42
Filter  43
Filter  44
Filter  45
Filter  46
Filter  47
Filter  48
Filter  49
Filter  50
Filter  51
Filter  52
Filter  53
Filter  54
Filter  55
Filter  56
Filter  57
Filter  58
Filter  59
Filter  60
Filter  61
Filter  62
Filter  63
Filter  64
Shape:  (114, 114, 64)

**ReLU**
S

Filter  203
Filter  204
Filter  205
Filter  206
Filter  207
Filter  208
Filter  209
Filter  210
Filter  211
Filter  212
Filter  213
Filter  214
Filter  215
Filter  216
Filter  217
Filter  218
Filter  219
Filter  220
Filter  221
Filter  222
Filter  223
Filter  224
Filter  225
Filter  226
Filter  227
Filter  228
Filter  229
Filter  230
Filter  231
Filter  232
Filter  233
Filter  234
Filter  235
Filter  236
Filter  237
Filter  238
Filter  239
Filter  240
Filter  241
Filter  242
Filter  243
Filter  244
Filter  245
Filter  246
Filter  247
Filter  248
Filter  249
Filter  250
Filter  251
Filter  252
Filter  253
Filter  254
Filter  255
Filter  256
Filter  257
Filter  258
Filter  259
Filter  260
Filter  261
Filter  262
Filter  263
Filter  264
Filter  265
Filter  266
Filter  267
Filter  268
Filter  269
Filter  270
Filter  271
Filter  272
Filter  273
Filter  274
Filter  275
Filter  276
Filter  277
Filter  278
Filter  279
Filter  280
Filter  281
Filter  282
Filter  283
Filter  284
Filter  285
Filt

### Flatten

In [88]:
last_feature_map.shape

(22, 22, 512)

In [89]:
def flatten(feature_map):
    total = 1
    for dim in feature_map.shape:
        total = total * dim
    
    flattened = feature_map.reshape(total,1)
    return flattened

In [90]:
flattened = flatten(last_feature_map)
flattened.shape

(247808, 1)

In [96]:
chosen_indices = np.random.randint(flattened.shape[0],size = 512)
chosen_indices.size

512

In [97]:
chosen_features = flattened[chosen_indices]

In [98]:
np.savetxt("chosen_features_CNN.csv",chosen_features,delimiter=",")

### LSTM

In [ ]:
#Activation Functions
#sigmoid
def sigmoid(X):
    return 1/(1+np.exp(-X))

#tanh activation
def tanh_activation(X):
    return np.tanh(X)

#softmax activation
def softmax(X):
    exp_X = np.exp(X)
    exp_X_sum = np.sum(exp_X,axis=1).reshape(-1,1)
    exp_X = exp_X/exp_X_sum
    return exp_X

#derivative of tanh
def tanh_derivative(X):
    return 1-(X**2)

In [ ]:
#initialize parameters
def initialize_parameters():
    #initialize the parameters with 0 mean and 0.01 standard deviation
    mean = 0
    std = 0.01
    
    #lstm cell weights
    forget_gate_weights = np.random.normal(mean,std,(input_units+hidden_units,hidden_units))
    input_gate_weights  = np.random.normal(mean,std,(input_units+hidden_units,hidden_units))
    output_gate_weights = np.random.normal(mean,std,(input_units+hidden_units,hidden_units))
    gate_gate_weights   = np.random.normal(mean,std,(input_units+hidden_units,hidden_units))
    
    #hidden to output weights (output cell)
    hidden_output_weights = np.random.normal(mean,std,(hidden_units,output_units))
    
    parameters = dict()
    parameters['fgw'] = forget_gate_weights
    parameters['igw'] = input_gate_weights
    parameters['ogw'] = output_gate_weights
    parameters['ggw'] = gate_gate_weights
    parameters['how'] = hidden_output_weights
    
    return parameters


In [ ]:
#single lstm cell
def lstm_cell(batch_dataset, prev_activation_matrix, prev_cell_matrix, parameters):
    #get parameters
    fgw = parameters['fgw']
    igw = parameters['igw']
    ogw = parameters['ogw']
    ggw = parameters['ggw']
    
    #concat batch data and prev_activation matrix
    concat_dataset = np.concatenate((batch_dataset,prev_activation_matrix),axis=1)
    
    #forget gate activations
    fa = np.matmul(concat_dataset,fgw)
    fa = sigmoid(fa)
    
    #input gate activations
    ia = np.matmul(concat_dataset,igw)
    ia = sigmoid(ia)
    
    #output gate activations
    oa = np.matmul(concat_dataset,ogw)
    oa = sigmoid(oa)
    
    #gate gate activations
    ga = np.matmul(concat_dataset,ggw)
    ga = tanh_activation(ga)
    
    #new cell memory matrix
    cell_memory_matrix = np.multiply(fa,prev_cell_matrix) + np.multiply(ia,ga)
    
    #current activation matrix
    activation_matrix = np.multiply(oa, tanh_activation(cell_memory_matrix))
    
    #lets store the activations to be used in back prop
    lstm_activations = dict()
    lstm_activations['fa'] = fa
    lstm_activations['ia'] = ia
    lstm_activations['oa'] = oa
    lstm_activations['ga'] = ga
    
    return lstm_activations,cell_memory_matrix,activation_matrix

In [ ]:
def output_cell(activation_matrix,parameters):
    #get hidden to output parameters
    how = parameters['how']
    
    #get outputs 
    output_matrix = np.matmul(activation_matrix,how)
    output_matrix = softmax(output_matrix)
    
    return output_matrix

### Get corresponding embeddings for the batch dataset

In [ ]:
def get_embeddings(batch_dataset,embeddings):
    embedding_dataset = np.matmul(batch_dataset,embeddings)
    return embedding_dataset

### Forward Propagation
![](https://github.com/navjindervirdee/neural-networks/blob/master/Recurrent%20Neural%20Network/LSTMForward.JPG?raw=true)

Function returns the intermediate ativations in the respective caches:
* LSTM Cache :- All lstm cell activation in every cell (fa,ia,ga,oa)
* Activation Cache : All activation (a0,a1,a2..)
* Cell Cache : All cell activations (c0,c1,c2..
* Embedding cache : Embeddings of each batch (e0,e1,e2..)
* Output Cache : All output (o1,o2,o3... )

In [ ]:
#forward propagation
def forward_propagation(batches,parameters,embeddings):
    #get batch size
    batch_size = batches[0].shape[0]
    
    #to store the activations of all the unrollings.
    lstm_cache = dict()                 #lstm cache
    activation_cache = dict()           #activation cache 
    cell_cache = dict()                 #cell cache
    output_cache = dict()               #output cache
    embedding_cache = dict()            #embedding cache 
    
    #initial activation_matrix(a0) and cell_matrix(c0)
    a0 = np.zeros([batch_size,hidden_units],dtype=np.float32)
    c0 = np.zeros([batch_size,hidden_units],dtype=np.float32)
    
    #store the initial activations in cache
    activation_cache['a0'] = a0
    cell_cache['c0'] = c0
    
    #unroll the names
    for i in range(len(batches)-1):
        #get first first character batch
        batch_dataset = batches[i]
        
        #get embeddings 
        batch_dataset = get_embeddings(batch_dataset,embeddings)
        embedding_cache['emb'+str(i)] = batch_dataset
        
        #lstm cell
        lstm_activations,ct,at = lstm_cell(batch_dataset,a0,c0,parameters)
        
        #output cell
        ot = output_cell(at,parameters)
        
        #store the time 't' activations in caches
        lstm_cache['lstm' + str(i+1)]  = lstm_activations
        activation_cache['a'+str(i+1)] = at
        cell_cache['c' + str(i+1)] = ct
        output_cache['o'+str(i+1)] = ot
        
        #update a0 and c0 to new 'at' and 'ct' for next lstm cell
        a0 = at
        c0 = ct
        
    return embedding_cache,lstm_cache,activation_cache,cell_cache,output_cache

### Calculate the Loss, Perplexity, and Accuracy
**Loss**
* Loss at time t = -sum(Y x log(d) + (1-Y) x log(1-pred)))/m
* Overall Loss = **∑**(Loss(t)) sum of all losses at each time step 't'

**Perplexity **
* Probability Product = **∏**(prob(pred_char)) for each char in name
* Perplexity = (1/probability_product) ^ (1/n) where n in number of chars in name

**Accuracy**
* Accuracy(t) = (Y==predictions,axis=1) for all time steps
* Accuracy = ((**∑**Acc(t))/batch_size)/n for all time steps, n is number of chars in name

In [ ]:
#calculate loss, perplexity and accuracy
def cal_loss_accuracy(batch_labels,output_cache):
    loss = 0  #to sum loss for each time step
    acc  = 0  #to sum acc for each time step 
    prob = 1  #probability product of each time step predicted char
    
    #batch size
    batch_size = batch_labels[0].shape[0]
    
    #loop through each time step
    for i in range(1,len(output_cache)+1):
        #get true labels and predictions
        labels = batch_labels[i]
        pred = output_cache['o'+str(i)]
        
        prob = np.multiply(prob,np.sum(np.multiply(labels,pred),axis=1).reshape(-1,1))
        loss += np.sum((np.multiply(labels,np.log(pred)) + np.multiply(1-labels,np.log(1-pred))),axis=1).reshape(-1,1)
        acc  += np.array(np.argmax(labels,1)==np.argmax(pred,1),dtype=np.float32).reshape(-1,1)
    
    #calculate perplexity loss and accuracy
    perplexity = np.sum((1/prob)**(1/len(output_cache)))/batch_size
    loss = np.sum(loss)*(-1/batch_size)
    acc  = np.sum(acc)/(batch_size)
    acc = acc/len(output_cache)
    
    return perplexity,loss,acc

### Calculate Output Cell Errors for each time step
* Output Error Cache :- to store output error for each time step
* Activation Error Cache : to store activation error for each time step

In [ ]:
#calculate output cell errors
def calculate_output_cell_error(batch_labels,output_cache,parameters):
    #to store the output errors for each time step
    output_error_cache = dict()
    activation_error_cache = dict()
    how = parameters['how']
    
    #loop through each time step
    for i in range(1,len(output_cache)+1):
        #get true and predicted labels
        labels = batch_labels[i]
        pred = output_cache['o'+str(i)]
        
        #calculate the output_error for time step 't'
        error_output = pred - labels
        
        #calculate the activation error for time step 't'
        error_activation = np.matmul(error_output,how.T)
        
        #store the output and activation error in dict
        output_error_cache['eo'+str(i)] = error_output
        activation_error_cache['ea'+str(i)] = error_activation
        
    return output_error_cache,activation_error_cache

In [ ]:
#calculate error for single lstm cell
def calculate_single_lstm_cell_error(activation_output_error,next_activation_error,next_cell_error,parameters,lstm_activation,cell_activation,prev_cell_activation):
    #activation error =  error coming from output cell and error coming from the next lstm cell
    activation_error = activation_output_error + next_activation_error
    
    #output gate error
    oa = lstm_activation['oa']
    eo = np.multiply(activation_error,tanh_activation(cell_activation))
    eo = np.multiply(np.multiply(eo,oa),1-oa)
    
    #cell activation error
    cell_error = np.multiply(activation_error,oa)
    cell_error = np.multiply(cell_error,tanh_derivative(tanh_activation(cell_activation)))
    #error also coming from next lstm cell 
    cell_error += next_cell_error
    
    #input gate error
    ia = lstm_activation['ia']
    ga = lstm_activation['ga']
    ei = np.multiply(cell_error,ga)
    ei = np.multiply(np.multiply(ei,ia),1-ia)
    
    #gate gate error
    eg = np.multiply(cell_error,ia)
    eg = np.multiply(eg,tanh_derivative(ga))
    
    #forget gate error
    fa = lstm_activation['fa']
    ef = np.multiply(cell_error,prev_cell_activation)
    ef = np.multiply(np.multiply(ef,fa),1-fa)
    
    #prev cell error
    prev_cell_error = np.multiply(cell_error,fa)
    
    #get parameters
    fgw = parameters['fgw']
    igw = parameters['igw']
    ggw = parameters['ggw']
    ogw = parameters['ogw']
    
    #embedding + hidden activation error
    embed_activation_error = np.matmul(ef,fgw.T)
    embed_activation_error += np.matmul(ei,igw.T)
    embed_activation_error += np.matmul(eo,ogw.T)
    embed_activation_error += np.matmul(eg,ggw.T)
    
    input_hidden_units = fgw.shape[0]
    hidden_units = fgw.shape[1]
    input_units = input_hidden_units - hidden_units
    
    #prev activation error
    prev_activation_error = embed_activation_error[:,input_units:]
    
    #input error (embedding error)
    embed_error = embed_activation_error[:,:input_units]
    
    #store lstm error
    lstm_error = dict()
    lstm_error['ef'] = ef
    lstm_error['ei'] = ei
    lstm_error['eo'] = eo
    lstm_error['eg'] = eg
    
    return prev_activation_error,prev_cell_error,embed_error,lstm_error

### Calculate Output Cell Derivatives for each time step

In [ ]:
#calculate output cell derivatives
def calculate_output_cell_derivatives(output_error_cache,activation_cache,parameters):
    #to store the sum of derivatives from each time step
    dhow = np.zeros(parameters['how'].shape)
    
    batch_size = activation_cache['a1'].shape[0]
    
    #loop through the time steps 
    for i in range(1,len(output_error_cache)+1):
        #get output error
        output_error = output_error_cache['eo' + str(i)]
        
        #get input activation
        activation = activation_cache['a'+str(i)]
        
        #cal derivative and summing up!
        dhow += np.matmul(activation.T,output_error)/batch_size
        
    return dhow

### Calculate LSTM CELL Derivatives for each time step

In [ ]:
#calculate derivatives for single lstm cell
def calculate_single_lstm_cell_derivatives(lstm_error,embedding_matrix,activation_matrix):
    #get error for single time step
    ef = lstm_error['ef']
    ei = lstm_error['ei']
    eo = lstm_error['eo']
    eg = lstm_error['eg']
    
    #get input activations for this time step
    concat_matrix = np.concatenate((embedding_matrix,activation_matrix),axis=1)
    
    batch_size = embedding_matrix.shape[0]
    
    #cal derivatives for this time step
    dfgw = np.matmul(concat_matrix.T,ef)/batch_size
    digw = np.matmul(concat_matrix.T,ei)/batch_size
    dogw = np.matmul(concat_matrix.T,eo)/batch_size
    dggw = np.matmul(concat_matrix.T,eg)/batch_size
    
    #store the derivatives for this time step in dict
    derivatives = dict()
    derivatives['dfgw'] = dfgw
    derivatives['digw'] = digw
    derivatives['dogw'] = dogw
    derivatives['dggw'] = dggw
    
    return derivatives

### Backward Propagation

* Apply chain rule and calculate the errors for each time step
* Store the deivatives in **derivatives** dict

In [ ]:
#backpropagation
def backward_propagation(batch_labels,embedding_cache,lstm_cache,activation_cache,cell_cache,output_cache,parameters):
    #calculate output errors 
    output_error_cache,activation_error_cache = calculate_output_cell_error(batch_labels,output_cache,parameters)
    
    #to store lstm error for each time step
    lstm_error_cache = dict()
    
    #to store embeding errors for each time step
    embedding_error_cache = dict()
    
    # next activation error 
    # next cell error  
    #for last cell will be zero
    eat = np.zeros(activation_error_cache['ea1'].shape)
    ect = np.zeros(activation_error_cache['ea1'].shape)
    
    #calculate all lstm cell errors (going from last time-step to the first time step)
    for i in range(len(lstm_cache),0,-1):
        #calculate the lstm errors for this time step 't'
        pae,pce,ee,le = calculate_single_lstm_cell_error(activation_error_cache['ea'+str(i)],eat,ect,parameters,lstm_cache['lstm'+str(i)],cell_cache['c'+str(i)],cell_cache['c'+str(i-1)])
        
        #store the lstm error in dict
        lstm_error_cache['elstm'+str(i)] = le
        
        #store the embedding error in dict
        embedding_error_cache['eemb'+str(i-1)] = ee
        
        #update the next activation error and next cell error for previous cell
        eat = pae
        ect = pce
    
    
    #calculate output cell derivatives
    derivatives = dict()
    derivatives['dhow'] = calculate_output_cell_derivatives(output_error_cache,activation_cache,parameters)
    
    #calculate lstm cell derivatives for each time step and store in lstm_derivatives dict
    lstm_derivatives = dict()
    for i in range(1,len(lstm_error_cache)+1):
        lstm_derivatives['dlstm'+str(i)] = calculate_single_lstm_cell_derivatives(lstm_error_cache['elstm'+str(i)],embedding_cache['emb'+str(i-1)],activation_cache['a'+str(i-1)])
    
    #initialize the derivatives to zeros 
    derivatives['dfgw'] = np.zeros(parameters['fgw'].shape)
    derivatives['digw'] = np.zeros(parameters['igw'].shape)
    derivatives['dogw'] = np.zeros(parameters['ogw'].shape)
    derivatives['dggw'] = np.zeros(parameters['ggw'].shape)
    
    #sum up the derivatives for each time step
    for i in range(1,len(lstm_error_cache)+1):
        derivatives['dfgw'] += lstm_derivatives['dlstm'+str(i)]['dfgw']
        derivatives['digw'] += lstm_derivatives['dlstm'+str(i)]['digw']
        derivatives['dogw'] += lstm_derivatives['dlstm'+str(i)]['dogw']
        derivatives['dggw'] += lstm_derivatives['dlstm'+str(i)]['dggw']
    
    return derivatives,embedding_error_cache

### Adam Optimizer
Using Exponentially Weighted Averages <br>
* Vdw = beta1 x Vdw + (1-beta1) x (dw)   
* Sdw = beta2 x Sdw + (1-beta2) x dw^2
* W = W - learning_rate x ( Vdw/ (sqrt(Sdw)+1e-7) )

In [ ]:
#update the parameters using adam optimizer
#adam optimization
def update_parameters(parameters,derivatives,V,S,t):
    #get derivatives
    dfgw = derivatives['dfgw']
    digw = derivatives['digw']
    dogw = derivatives['dogw']
    dggw = derivatives['dggw']
    dhow = derivatives['dhow']
    
    #get parameters
    fgw = parameters['fgw']
    igw = parameters['igw']
    ogw = parameters['ogw']
    ggw = parameters['ggw']
    how = parameters['how']
    
    #get V parameters
    vfgw = V['vfgw']
    vigw = V['vigw']
    vogw = V['vogw']
    vggw = V['vggw']
    vhow = V['vhow']
    
    #get S parameters
    sfgw = S['sfgw']
    sigw = S['sigw']
    sogw = S['sogw']
    sggw = S['sggw']
    show = S['show']
    
    #calculate the V parameters from V and current derivatives
    vfgw = (beta1*vfgw + (1-beta1)*dfgw)
    vigw = (beta1*vigw + (1-beta1)*digw)
    vogw = (beta1*vogw + (1-beta1)*dogw)
    vggw = (beta1*vggw + (1-beta1)*dggw)
    vhow = (beta1*vhow + (1-beta1)*dhow)
    
    #calculate the S parameters from S and current derivatives
    sfgw = (beta2*sfgw + (1-beta2)*(dfgw**2))
    sigw = (beta2*sigw + (1-beta2)*(digw**2))
    sogw = (beta2*sogw + (1-beta2)*(dogw**2))
    sggw = (beta2*sggw + (1-beta2)*(dggw**2))
    show = (beta2*show + (1-beta2)*(dhow**2))
    
    #update the parameters
    fgw = fgw - learning_rate*((vfgw)/(np.sqrt(sfgw) + 1e-6))
    igw = igw - learning_rate*((vigw)/(np.sqrt(sigw) + 1e-6))
    ogw = ogw - learning_rate*((vogw)/(np.sqrt(sogw) + 1e-6))
    ggw = ggw - learning_rate*((vggw)/(np.sqrt(sggw) + 1e-6))
    how = how - learning_rate*((vhow)/(np.sqrt(show) + 1e-6))
    
    #store the new weights
    parameters['fgw'] = fgw
    parameters['igw'] = igw
    parameters['ogw'] = ogw
    parameters['ggw'] = ggw
    parameters['how'] = how
    
    #store the new V parameters
    V['vfgw'] = vfgw 
    V['vigw'] = vigw 
    V['vogw'] = vogw 
    V['vggw'] = vggw
    V['vhow'] = vhow
    
    #store the s parameters
    S['sfgw'] = sfgw 
    S['sigw'] = sigw 
    S['sogw'] = sogw 
    S['sggw'] = sggw
    S['show'] = show
    
    return parameters,V,S    

### Update the embeddings

In [ ]:
#update the Embeddings
def update_embeddings(embeddings,embedding_error_cache,batch_labels):
    #to store the embeddings derivatives
    embedding_derivatives = np.zeros(embeddings.shape)
    
    batch_size = batch_labels[0].shape[0]
    
    #sum the embedding derivatives for each time step
    for i in range(len(embedding_error_cache)):
        embedding_derivatives += np.matmul(batch_labels[i].T,embedding_error_cache['eemb'+str(i)])/batch_size
    
    #update the embeddings
    embeddings = embeddings - learning_rate*embedding_derivatives
    return embeddings

### Functions to Initialize the V and S parameters for Adam Optimizer

In [ ]:
def initialize_V(parameters):
    Vfgw = np.zeros(parameters['fgw'].shape)
    Vigw = np.zeros(parameters['igw'].shape)
    Vogw = np.zeros(parameters['ogw'].shape)
    Vggw = np.zeros(parameters['ggw'].shape)
    Vhow = np.zeros(parameters['how'].shape)
    
    V = dict()
    V['vfgw'] = Vfgw
    V['vigw'] = Vigw
    V['vogw'] = Vogw
    V['vggw'] = Vggw
    V['vhow'] = Vhow
    return V

def initialize_S(parameters):
    Sfgw = np.zeros(parameters['fgw'].shape)
    Sigw = np.zeros(parameters['igw'].shape)
    Sogw = np.zeros(parameters['ogw'].shape)
    Sggw = np.zeros(parameters['ggw'].shape)
    Show = np.zeros(parameters['how'].shape)
    
    S = dict()
    S['sfgw'] = Sfgw
    S['sigw'] = Sigw
    S['sogw'] = Sogw
    S['sggw'] = Sggw
    S['show'] = Show
    return S

### Train Function

1. Initialize Parameters
2. Forward Propagation
3. Calculate Loss, Perplexity and Accuracy
4. Backward Propagation
5. Update the Parameters and Embeddings

Batch Size = 20
Repeat the steps 2-5 for each batch!

In [ ]:
#train function
def train(train_dataset,iters=1000,batch_size=20):
    #initalize the parameters
    parameters = initialize_parameters()
    
    #initialize the V and S parameters for Adam
    V = initialize_V(parameters)
    S = initialize_S(parameters)
    
    #generate the random embeddings
    embeddings = np.random.normal(0,0.01,(len(vocab),input_units))
    
    #to store the Loss, Perplexity and Accuracy for each batch
    J = []
    P = []
    A = []
    
    
    for step in range(iters):
        #get batch dataset
        index = step%len(train_dataset)
        batches = train_dataset[index]
        
        #forward propagation
        embedding_cache,lstm_cache,activation_cache,cell_cache,output_cache = forward_propagation(batches,parameters,embeddings)
        
        #calculate the loss, perplexity and accuracy
        perplexity,loss,acc = cal_loss_accuracy(batches,output_cache)
        
        #backward propagation
        derivatives,embedding_error_cache = backward_propagation(batches,embedding_cache,lstm_cache,activation_cache,cell_cache,output_cache,parameters)
        
        #update the parameters
        parameters,V,S = update_parameters(parameters,derivatives,V,S,step)
        
        #update the embeddings
        embeddings = update_embeddings(embeddings,embedding_error_cache,batches)
        
        
        J.append(loss)
        P.append(perplexity)
        A.append(acc)
        
        #print loss, accuracy and perplexity
        if(step%1000==0):
            print("For Single Batch :")
            print('Step       = {}'.format(step))
            print('Loss       = {}'.format(round(loss,2)))
            print('Perplexity = {}'.format(round(perplexity,2)))
            print('Accuracy   = {}'.format(round(acc*100,2)))
            print()
    
    return embeddings, parameters,J,P,A

## Let's Train
* Will take around 5-10 mins on CPU

In [ ]:
flattened

In [ ]:
output_units = 1004

In [ ]:
embeddings,parameters,J,P,A = train(flattened,iters=8001)

In [ ]:
avg_loss = list()
avg_acc = list()
avg_perp = list()
i = 0
while(i<len(J)):
    avg_loss.append(np.mean(J[i:i+30]))
    avg_acc.append(np.mean(A[i:i+30]))
    avg_perp.append(np.mean(P[i:i+30]))
    i += 30

plt.plot(list(range(len(avg_loss))),avg_loss)
plt.xlabel("x")
plt.ylabel("Loss (Avg of 30 batches)")
plt.title("Loss Graph")
plt.show()

plt.plot(list(range(len(avg_perp))),avg_perp)
plt.xlabel("x")
plt.ylabel("Perplexity (Avg of 30 batches)")
plt.title("Perplexity Graph")
plt.show()

plt.plot(list(range(len(avg_acc))),avg_acc)
plt.xlabel("x")
plt.ylabel("Accuracy (Avg of 30 batches)")
plt.title("Accuracy Graph")
plt.show()    